<a href="https://colab.research.google.com/github/claubermartins/Stock-market-prediction/blob/main/stock_market_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install inflection

#**1-Importando bibliotecas**

In [2]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import EarlyStopping 
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import tensorflow as tf
import math
import seaborn as sns
import datetime
import inflection
from IPython.core.display import HTML

#**2-Funções auxiliares para o Jupyter Notebook**

In [3]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()

In [4]:
jupyter_settings()

Populating the interactive namespace from numpy and matplotlib


#**2.1Método de importação de arquivos do Google Colab**

In [5]:
from google.colab import files
upload = files.upload()
print("downloaded files: ")
print(*upload, sep = "\n")

Saving PETR4-teste.csv to PETR4-teste.csv
Saving PETR4-treinamento.csv to PETR4-treinamento.csv
downloaded files: 
PETR4-teste.csv
PETR4-treinamento.csv


#**3-Tratamento dos dados**

##**3.1-Importando os dados**

In [7]:
base = pd.read_csv('PETR4-treinamento.csv')
base = base.drop('Date', axis = 1)

In [9]:
#valores estatísticos
base.describe()

,Open,High,Low,Close,Adj Close,Volume
count,5030.000000,5030.000000,5030.000000,5030.000000,5030.000000,5.030000e+03
mean,17.891079,18.151007,17.607981,17.861287,14.436626,6.238688e+08
std,9.609917,9.761328,9.434506,9.571652,7.927764,4.182380e+09
min,4.200000,4.270000,4.120000,4.200000,2.934877,0.000000e+00
25%,9.190000,9.375312,8.963438,9.192500,7.050538,1.986212e+07
50%,17.464999,17.799999,17.150000,17.450001,14.825717,3.011960e+07
75%,24.077500,24.398750,23.670000,24.015000,19.731367,4.701040e+07
max,67.500000,67.500000,51.950001,52.509998,39.535816,7.356416e+10


In [10]:
base.dtypes

Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume       float64
dtype: object

##**3.2-Tratando os Valores Faltantes**

In [11]:
#valores faltantes
base.isna().sum()

Open         3
High         3
Low          3
Close        3
Adj Close    3
Volume       3
dtype: int64

In [13]:
#encontrando onde os valores faltantes estão
base.loc[pd.isnull(base['Open'])]
base.loc[pd.isnull(base['High'])]
base.loc[pd.isnull(base['Low'])]
base.loc[pd.isnull(base['Close'])]
base.loc[pd.isnull(base['Adj Close'])]
base.loc[pd.isnull(base['Volume'])]

,Open,High,Low,Close,Adj Close,Volume
1260,NaN,NaN,NaN,NaN,NaN,NaN
4568,NaN,NaN,NaN,NaN,NaN,NaN
4827,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
#preenchendo os valores nulos com o valor médio
imputer = SimpleImputer(missing_values=np.nan, strategy='mean',verbose=0)
imputer = imputer.fit(base.iloc[:,0:6])
base.iloc[:,0:6] = imputer.transform(base.iloc[:,0:6])

In [15]:
#verificando novamente se ainda temos valores faltantes
base.loc[pd.isnull(base['Open'])]
base.loc[pd.isnull(base['High'])]
base.loc[pd.isnull(base['Low'])]
base.loc[pd.isnull(base['Close'])]
base.loc[pd.isnull(base['Adj Close'])]
base.loc[pd.isnull(base['Volume'])]

,Open,High,Low,Close,Adj Close,Volume


#**3.3-Tratamento dos valores nulos (iguais a 0)**

In [16]:
#valores nulos
(base == 0).sum()

Open           0
High           0
Low            0
Close          0
Adj Close      0
Volume       146
dtype: int64

In [17]:
#Encontrando os valores nulos
base.loc[base['Volume'] == 0]

,Open,High,Low,Close,Adj Close,Volume
15,5.163000,5.163000,5.163000,5.163000,3.463490,0.0
44,6.038000,6.038000,6.038000,6.038000,4.050466,0.0
45,6.038000,6.038000,6.038000,6.038000,4.050466,0.0
49,6.213000,6.213000,6.213000,6.213000,4.167862,0.0
73,5.900000,5.900000,5.900000,5.900000,3.957891,0.0
78,5.100000,5.100000,5.100000,5.100000,3.421229,0.0
84,5.376000,5.376000,5.376000,5.376000,3.606377,0.0
110,5.790000,5.790000,5.790000,5.790000,3.884100,0.0
122,6.438000,6.438000,6.438000,6.438000,4.318798,0.0
177,7.112500,7.112500,7.112500,7.112500,4.771273,0.0


In [18]:
#preenchendo os valores nulos com o valor médio
imputer = SimpleImputer(missing_values=0, strategy='mean',verbose=0)
imputer = imputer.fit(base.iloc[:,0:6])
base.iloc[:,0:6] = imputer.transform(base.iloc[:,0:6])

In [19]:
#Encontrando os valores nulos
base.loc[base['Volume'] == 0]

,Open,High,Low,Close,Adj Close,Volume


In [17]:
#base = base.dropna()

In [18]:
#base.sample()

#**-Descrição dos dados** IGNORE ESTA CÉLULA

In [ ]:
#Tendencia central - mean, median
ct1 = pd.DataFrame(base.apply(np.mean)).T #T transpondo 
ct2 = pd.DataFrame(base.apply(np.median)).T
#Dispersão - std(desvio padrão), min, max, range
d1 = pd.DataFrame(base.apply(np.std)).T
d2 = pd.DataFrame(base.apply(min)).T
d3 = pd.DataFrame(base.apply(max)).T
d4 = pd.DataFrame(base.apply(lambda x: x.max() - x.min())).T
#concatenate
m = pd.concat([d2,d3,d4,ct1,ct2,d1]).T.reset_index()
m.columns = ['attributes','min','max','range', 'mean', 'median', 'std']
m

,attributes,min,max,range,mean,median,std
0,Open,4.200000,3.743000e+01,3.323000e+01,1.933303e+01,1.909000e+01,6.833255e+00
1,High,4.270000,3.750000e+01,3.323000e+01,1.959720e+01,1.937000e+01,6.867448e+00
2,Low,4.120000,3.707000e+01,3.295000e+01,1.904492e+01,1.878000e+01,6.786986e+00
3,Close,4.200000,3.750000e+01,3.330000e+01,1.930418e+01,1.910000e+01,6.824516e+00
4,Adj Close,3.896351,3.054497e+01,2.664862e+01,1.705664e+01,1.670628e+01,5.745576e+00
5,Volume,0.000000,6.989506e+08,6.989506e+08,4.115402e+07,3.508690e+07,3.058053e+07


In [ ]:
base.shape[0]

2478

#**4-Treinamento da rede**

In [20]:
#normalizando a base de dados com o MinMaxScaler
base_treinamento = base.iloc[:, 0:6].values
normalizador = MinMaxScaler(feature_range=(0,1))
base_treinamento_normalizada = normalizador.fit_transform(base_treinamento)

In [21]:
#Comparando a predição com o preço real
previsores = []
preco_real = []
for i in range(90, 4785):
    previsores.append(base_treinamento_normalizada[i-90:i, 0:6])
    preco_real.append(base_treinamento_normalizada[i, 0])
previsores, preco_real = np.array(previsores), np.array(preco_real)


In [22]:
#estrutura da rede neural 
regressor = Sequential()
regressor.add(LSTM(units = 100, return_sequences = True, input_shape = (previsores.shape[1], 6)))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.3))

regressor.add(Dense(units = 1, activation = 'sigmoid'))

regressor.compile(optimizer = 'RMSprop', loss = 'mean_squared_error',
                  metrics = ['mean_absolute_error'])

es = EarlyStopping(monitor = 'loss', min_delta = 1e-15, patience = 20, verbose = 1)
#opt = tf.keras.optimizers.Adam(amsgrad=True)
#regressor.compile(optimizer = opt, loss = 'mean_squared_error',
#                  metrics = ['mean_absolute_error'])
#opt = tf.keras.optimizers.RMSprop(centered=True)
#regressor.compile(optimizer = opt, loss = 'mean_squared_error',
#                 metrics = ['mean_absolute_error'])

regressor.fit(previsores, preco_real, epochs = 600, batch_size = 32,
              callbacks = [es])

Epoch 1/600
147/147 [==============================] - 34s 187ms/step - loss: 0.0189 - mean_absolute_error: 0.0978
Epoch 2/600
147/147 [==============================] - 28s 188ms/step - loss: 0.0026 - mean_absolute_error: 0.0387
Epoch 3/600
147/147 [==============================] - 28s 190ms/step - loss: 0.0020 - mean_absolute_error: 0.0310
Epoch 4/600
147/147 [==============================] - 29s 194ms/step - loss: 0.0016 - mean_absolute_error: 0.0293
Epoch 5/600
147/147 [==============================] - 29s 194ms/step - loss: 0.0012 - mean_absolute_error: 0.0270
Epoch 6/600
147/147 [==============================] - 29s 195ms/step - loss: 0.0011 - mean_absolute_error: 0.0245
Epoch 7/600
147/147 [==============================] - 29s 196ms/step - loss: 0.0016 - mean_absolute_error: 0.0244
Epoch 8/600
147/147 [==============================] - 29s 196ms/step - loss: 9.7743e-04 - mean_absolute_error: 0.0220
Epoch 9/600
147/147 [==============================] - 29s 196ms/step - loss

#**5-Teste da rede**

##**5.1-Importando os dados para teste**

In [23]:
#criando a base de dados teste
base_teste = pd.read_csv('PETR4-teste.csv')
base_teste = base_teste.drop('Date', axis = 1)

In [24]:
#valores estatísticos
base_teste.describe()

,Open,High,Low,Close,Adj Close,Volume
count,22.000000,22.000000,22.000000,22.000000,22.000000,2.200000e+01
mean,29.765454,29.995455,29.448636,29.727273,29.725069,4.276695e+07
std,0.797500,0.726693,0.810428,0.776624,0.776566,1.489164e+07
min,28.280001,28.639999,27.670000,28.030001,28.027924,2.539750e+07
25%,29.207500,29.715001,28.900000,29.292500,29.290328,3.257412e+07
50%,29.765000,29.935001,29.545000,29.750000,29.747795,3.733660e+07
75%,30.420000,30.575001,30.189999,30.382500,30.380248,4.797648e+07
max,30.879999,31.240000,30.469999,30.809999,30.807716,8.184400e+07


##**5.2-Tratando os Valores Faltantes**

In [25]:
#valores faltantes
base_teste.isna().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [26]:
#encontrando onde os valores faltantes estão
base_teste.loc[pd.isnull(base_teste['Open'])]
base_teste.loc[pd.isnull(base_teste['High'])]
base_teste.loc[pd.isnull(base_teste['Low'])]
base_teste.loc[pd.isnull(base_teste['Close'])]
base_teste.loc[pd.isnull(base_teste['Adj Close'])]
base_teste.loc[pd.isnull(base_teste['Volume'])]

,Open,High,Low,Close,Adj Close,Volume


In [27]:
#preenchendo os valores faltantes com o valor médio
imputer = SimpleImputer(missing_values=np.nan, strategy='mean',verbose=0)
imputer = imputer.fit(base_teste.iloc[:,0:6])
base_teste.iloc[:,0:6] = imputer.transform(base_teste.iloc[:,0:6])

In [29]:
#verificando novamente se ainda temos valores faltantes
base_teste.loc[pd.isnull(base_teste['Open'])]
base_teste.loc[pd.isnull(base_teste['High'])]
base_teste.loc[pd.isnull(base_teste['Low'])]
base_teste.loc[pd.isnull(base_teste['Close'])]
base_teste.loc[pd.isnull(base_teste['Adj Close'])]
base_teste.loc[pd.isnull(base_teste['Volume'])]

,Open,High,Low,Close,Adj Close,Volume


#**5.3-Tratamento dos valores nulos (iguais a 0)**

In [27]:
#valores nulos
(base_teste == 0).sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [31]:
#Encontrando os valores nulos
base_teste.loc[base_teste['Volume'] == 0]

,Open,High,Low,Close,Adj Close,Volume


In [220]:
#preenchendo os valores nulos com o valor médio
imputer = SimpleImputer(missing_values=0, strategy='mean',verbose=0)
imputer = imputer.fit(base_teste.iloc[:,0:6])
base_teste.iloc[:,0:6] = imputer.transform(base_teste.iloc[:,0:6])

In [221]:
#Verificando novamente se tem valores nulos
base_teste.loc[base_teste['Volume'] == 0]

,Open,High,Low,Close,Adj Close,Volume


##**5.4-Preparando a estrutura para teste**

In [28]:
#concatenando a base de dados teste com o base de treinamento
preco_real_teste = base_teste.iloc[:, 3:4].values
frames = [base, base_teste]
base_completa = pd.concat(frames)

In [29]:
#pegando os 90 registros anteriores para percorrer a base teste e colocando no
#no formato np.array
entradas = base_completa[len(base_completa) - len(base_teste) - 90:].values
entradas = normalizador.transform(entradas)
Percorrer_teste = []
for i in range(90, 112): 
    Percorrer_teste.append(entradas[i-90:i, 0:6])
Percorrer_teste = np.array(Percorrer_teste)

In [30]:
#resultado da predição no formato MinMaxScaler
previsoes = regressor.predict(Percorrer_teste)

In [31]:
#convertendo para a escala MinMaxScaler 
normalizador_previsao = MinMaxScaler(feature_range=(0,1))
normalizador_previsao.fit_transform(base_treinamento[:,3:4])

array([[0.02794453],
       [0.02678535],
       [0.02639205],
       ...,
       [0.55288762],
       [0.54481476],
       [0.53777688]])

In [32]:
#visualisando a previsão com no formato original
previsoes = normalizador_previsao.inverse_transform(previsoes)

##**6-Análise dos resuldatos**

In [ ]:
#Visualizando o gráfico da Previsão da rede
plt.plot(preco_real_teste, color = 'red', label = 'Preço real')
plt.plot(previsoes, color = 'blue', label = 'Previsões')
plt.title('Previsão do preço das ações')
plt.xlabel('Tempo em dias')
plt.ylabel('Preço das ações')
plt.legend()
plt.show()

In [ ]:
previsoes.mean()

In [ ]:
preco_real_teste.mean()